In [1]:
import numpy as np
#a1=np.zeros((3,2))
#a2=np.ones((2,3))
#np.random.rand(3,4)
#axis 指的是被“压缩掉”的维度。如果为1则列压缩，为0行压缩
#mask = np.triu(np.ones_like(A), k=1)  # 布尔或 float 掩码

# 方法1：直接用 np.where
#A_masked = np.where(mask == 1, -np.inf, A)

#单个attention实现

In [2]:
#单个attention实现
def init_qkv(X,l,d):
    d=8
    l=3
    X=np.random.rand(l,d)#3代表几个token，8代表embedding层
    W1=np.random.rand(d,d)
    W2=np.random.rand(d,d)
    W3=np.random.rand(d,d)
    Q=X@W1
    K=X@W2
    V=X@W3
    return Q,K,V
##3个都是3*8的
#自定义的sigmoid



def softmax(X,a=-1):#1代表按照行求均值 ,axis=-1表示最后一个维度
    x_shift=X-np.max(X,axis=a,keepdims=True)##同时减去一个最大值，保持整体不变
    x_shift=np.exp(x_shift)
    return x_shift/np.sum(x_shift,axis=a,keepdims=True)


def attention_solo(Q,K,V):
    if Q.ndim == 3:
        KK=K.transpose(0,2,1)
        score=softmax((Q@KK)/(np.sqrt(Q.shape[2])),-1)
    else:
        score=softmax((Q@K.T)/(np.sqrt(Q.shape[1])),-1)
    attention=score@V
    return attention


def muti_att(n_heads,Q,K,V):
    W1=np.random.rand(Q.shape[1],Q.shape[1])
    
    QQ=np.array(Q).reshape(Q.shape[0],n_heads,-1).transpose(1, 0, 2)                #需要注意attention是针对所有token进行操作
    KK=np.array(K).reshape(K.shape[0],n_heads,-1).transpose(1, 0, 2)
    VV=np.array(V).reshape(V.shape[0],n_heads,-1).transpose(1, 0, 2)
    attention_plus=attention_solo(QQ,KK,VV)


    attention_plus=attention_plus.transpose(1, 0, 2)
    attention_plus=attention_plus.reshape(attention_plus.shape[0],-1)
    final_attention=attention_plus@W1

    return final_attention
def positional_encoding(seq_len, d_model):

    PE = np.zeros((seq_len, d_model))
    position = np.arange(seq_len).reshape(-1, 1)  # (seq_len, 1)
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))  # (d_model//2,)  #numpy的广播机制是从右往左对齐的
    
    # 偶数列：sin
    PE[:, 0::2] = np.sin(position * div_term)
    # 奇数列：cos
    PE[:, 1::2] = np.cos(position * div_term)
    
    return PE
def lay_norm(X,a,b):
    mean=X.mean(axis=-1,keepdims=True)
    var = X.var(axis=-1, keepdims=True)  
    
    eps = 1e-5
    X_norm = ((X - mean) / np.sqrt(var + eps))*a+b
    return X_norm



In [3]:
#ffn是一个二维度的线性变换


def lay_norm(X,a=1.0,b=0.0):
    mean=X.mean(axis=-1,keepdims=True)
    var = X.var(axis=-1, keepdims=True)  
    
    eps = 1e-5
    X_norm = ((X - mean) / np.sqrt(var + eps))*a+b
    return X_norm

def softmax(X,a=-1):#1代表按照行求均值 ,axis=-1表示最后一个维度
    x_shift=X-np.max(X,axis=a,keepdims=True)##同时减去一个最大值，保持整体不变
    x_shift=np.exp(x_shift)
    return x_shift/np.sum(x_shift,axis=a,keepdims=True)

class Encoder:
    def __init__(self,X,sen_len,d_model,n_heads):
        self.n_heads=n_heads
        self.X=X
        self.sen_len=sen_len
        self.d_model=d_model

        self.Wq = np.random.randn(d_model, d_model)
        self.Wk = np.random.randn(d_model, d_model)
        self.Wv = np.random.randn(d_model, d_model)
        self.Wo = np.random.randn(d_model, d_model) 

        hidden_dim = d_model * 4
        self.ffn_w1 = np.random.randn(d_model, hidden_dim)
        self.ffn_w2 = np.random.randn(hidden_dim, d_model)

    def init_qkv(self,X):
        Q=X@self.Wq
        K=X@self.Wk
        V=X@self.Wv
        return Q,K,V

    def attention_solo(self,Q,K,V):
        if Q.ndim == 3:
            KK=K.transpose(0,2,1)
            score=softmax((Q@KK)/(np.sqrt(Q.shape[2])),-1)
        else:
            score=softmax((Q@K.T)/(np.sqrt(Q.shape[1])),-1)
        attention=score@V
        return attention

    def multi_head_attention(self,X):
        Q, K, V = self.init_qkv(X) 
        QQ=np.array(Q).reshape(Q.shape[0],self.n_heads,-1).transpose(1, 0, 2)                #需要注意attention是针对所有token进行操作
        KK=np.array(K).reshape(K.shape[0],self.n_heads,-1).transpose(1, 0, 2)
        VV=np.array(V).reshape(V.shape[0],self.n_heads,-1).transpose(1, 0, 2)
        attention_plus=self.attention_solo(QQ,KK,VV)


        attention_plus=attention_plus.transpose(1, 0, 2)
        attention_plus=attention_plus.reshape(attention_plus.shape[0],-1)
        final_attention=attention_plus@self.Wo

        return final_attention
    

    def positional_encoding(self):

        PE = np.zeros((self.sen_len, self.d_model))
        position = np.arange(self.sen_len).reshape(-1, 1)  # (seq_len, 1)
        div_term = np.exp(np.arange(0, self.d_model, 2) * -(np.log(10000.0) / self.d_model))  # (d_model//2,)  #numpy的广播机制是从右往左对齐的
        
        # 偶数列：sin
        PE[:, 0::2] = np.sin(position * div_term)
        # 奇数列：cos
        PE[:, 1::2] = np.cos(position * div_term)
        
        return PE
    def ffn(self, X):
        hidden = np.maximum(0, X @ self.ffn_w1)  # ReLU, (seq_len, hidden_dim)
        output = hidden @ self.ffn_w2  
        return output


    def forward(self):
        # Step 1: Add positional encoding
        X_pe = self.X + self.positional_encoding()  # (seq_len, d_model)

        # Step 2: Multi-head attention + residual connection
        attn_out = self.multi_head_attention(X_pe)  # (seq_len, d_model)
        x1 = lay_norm(X_pe + attn_out)              # LayerNorm after residual

        # Step 3: Feed-forward network + residual connection
        ffn_out = self.ffn(x1)                      # (seq_len, d_model)
        encoder_output = lay_norm(x1 + ffn_out)     # Final output
        return encoder_output


In [4]:
X = np.random.randn(5, 64)
encoder = Encoder(X, sen_len=5, d_model=64, n_heads=8)
output = encoder.forward()
print("Output shape:", output.shape)  # (5, 64)

Output shape: (5, 64)


decoder

In [5]:
class Decoder:
    def __init__(self,X,en_output,sen_len,de_model,nums_head):
        self.X=X
        self.en_output=en_output
        self.sen_len=X.shape[0]
        self.de_model=de_model
        self.nums_head=nums_head
        #ENCODER——decoder的参数
        self.enco_q=np.random.rand(de_model,de_model)
        self.enco_k=np.random.rand(de_model,de_model)
        self.W_v_ed = np.random.randn(de_model, de_model)
        self.W_o_ed = np.random.randn(de_model, de_model)


        self.ffn1=np.random.rand(de_model,de_model*4)
        self.ffn2=np.random.rand(de_model*4,de_model)

        self.q=np.random.rand(de_model,de_model)
        self.k=np.random.rand(de_model,de_model)
        self.v=np.random.rand(de_model,de_model)
        self.o=np.random.rand(de_model,de_model)
        

    def init_qkv(self,X):
        Q=X@self.q
        K=X@self.k
        V=X@self.v
        return Q,K,V
    def mask(self,sen_len):
        A = np.zeros((sen_len, sen_len))
        mask = np.triu(np.ones_like(A), k=1)  # 布尔或 float 掩码
        A_masked = np.where(mask == 1, -np.inf, A)
        loss_mask=np.where(mask == 1,1,0)

        return A_masked,loss_mask
    
    def split_heads(self,X):
        QQ=np.array(X).reshape(X.shape[0],self.nums_head,-1)              #需要注意attention是针对所有token进行操作
        return QQ
        
    def attention_mask_multi(self,Q,K,V,mask=None):
        Q=self.split_heads(Q)
        K=self.split_heads(K)
        V=self.split_heads(V)
        QQ=Q.transpose(1,0,2)
        kk=K.transpose(1,2,0)
        att=QQ@kk
        if mask is not None:
            att_mask=att+mask
        else:
            att_mask = att
        score_mask=softmax((att_mask)/(np.sqrt(Q.shape[2])),-1)
        vv=V.transpose(1,0,2)
        att_v=score_mask@vv
        att_final=att_v.transpose(1,0,2)
        att_final=att_final.reshape(att_final.shape[0],-1)

        return att_final
    

        #self.W_o_ed = np.random.randn(de_model, de_model)
    def cross_attention(self,enc,dec,mask=None):
        Q_dec=dec@self.enco_q

        K_dec=enc@self.enco_k
        V_dec=enc@self.W_v_ed
 
        final=self.attention_mask_multi(Q_dec,K_dec,V_dec)
        return final


    def positional_encoding(self):

        PE = np.zeros((self.sen_len, self.de_model))
        position = np.arange(self.sen_len).reshape(-1, 1)  # (seq_len, 1)
        div_term = np.exp(np.arange(0, self.de_model, 2) * -(np.log(10000.0) / self.de_model))  # (d_model//2,)  #numpy的广播机制是从右往左对齐的
        
        # 偶数列：sin
        PE[:, 0::2] = np.sin(position * div_term)
        # 奇数列：cos
        PE[:, 1::2] = np.cos(position * div_term)

        return PE

    def ffn(self,X):
        relu=np.maximum(0, X @ self.ffn1) 
        outputt=relu@self.ffn2
        return outputt
    
    def forward(self):
        X_1=self.positional_encoding()+self.X
        Q,K,V=self.init_qkv(X_1)
        mask,loss=self.mask(self.sen_len)
        atten1=(self.attention_mask_multi(Q,K,V,mask))@self.o
        de1_out=lay_norm(atten1+X_1)


        att2=self.cross_attention(self.en_output,de1_out,None)@self.W_o_ed
        de2_out=lay_norm(att2+de1_out)

        fin=lay_norm(self.ffn(de2_out)+de2_out)

        return fin

In [6]:

seq_len = 5
d_model = 8
num_heads = 2

# 模拟输入
X = np.random.randn(5, 64)          # decoder input embeddings

encoder = Encoder(X, sen_len=5, d_model=64, n_heads=8)
en_output = encoder.forward()

decoder = Decoder(X, en_output, 5, 64, 8)
output = decoder.forward()


In [7]:
import numpy as np
def softmax(x, axis=-1):
    x_max = np.max(x, axis=axis, keepdims=True)
    exp_x = np.exp(x - x_max)
    return exp_x / np.sum(exp_x, axis=axis, keepdims=True)

class Transformer:
    def __init__(self, vob_size, seq_len, d_model, num_heads, en_layer, de_layer):
        self.vob_size = vob_size
        self.seq_len = seq_len
        self.d_model = d_model
        self.num_heads = num_heads
        self.en_layer = en_layer
        self.de_layer = de_layer
        self.output_proj = np.random.randn(d_model, vob_size)  # 输出投影

    def forward(self, src, tgt):
        """
        src: (src_len, d_model) —— encoder input embeddings
        tgt: (tgt_len, d_model) —— decoder input embeddings (e.g., shifted target)
        """
        # ===== Encoder stack =====
        enc_output = src
        for _ in range(self.en_layer):
            encoder = Encoder(enc_output, self.seq_len, self.d_model, self.num_heads)
            enc_output = encoder.forward()  # 返回 (src_len, d_model)

        # ===== Decoder stack =====
        dec_output = tgt
        for _ in range(self.de_layer):
            decoder = Decoder(dec_output, enc_output, self.seq_len, self.d_model, self.num_heads)
            dec_output = decoder.forward()  # 返回 (tgt_len, d_model)

        logits = dec_output @ self.output_proj  # (tgt_len, vob_size)
        output = softmax(logits, axis=-1)
        return output

In [8]:
# 设置随机种子（可复现）
np.random.seed(4)

# 超参数
vob_size = 1000      # 词表大小
src_len = 6          # 源序列长度
tgt_len = 5          # 目标序列长度
d_model = 32         # 模型维度
num_heads = 4
en_layer = 2
de_layer = 2

# 模拟 embedding 输入（通常由 Embedding 层生成）
src_embed = np.random.randn(src_len, d_model)  # (6, 32)
tgt_embed = np.random.randn(tgt_len, d_model)  # (5, 32)

# 创建 Transformer 模型
model = Transformer(
    vob_size=vob_size,
    seq_len=max(src_len, tgt_len),  # 注意：这里简化处理，实际应分别处理 src/tgt len
    d_model=d_model,
    num_heads=num_heads,
    en_layer=en_layer,
    de_layer=de_layer
)

# 前向传播
output = model.forward(src_embed, tgt_embed)

# 打印结果
print("Source input shape:", src_embed.shape)      # (6, 32)
print("Target input shape:", tgt_embed.shape)      # (5, 32)
print("Model output shape:", output.shape)         # (5, 1000)
print("Output sum per token (should be ～1):", np.round(output.sum(axis=-1), decimals=6))

Source input shape: (6, 32)
Target input shape: (5, 32)
Model output shape: (5, 1000)
Output sum per token (should be ～1): [1. 1. 1. 1. 1.]


In [62]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

    



class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length) :
        super(PositionalEncoding, self).__init__()
        PE=torch.zeros(max_seq_length,d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        ev=position*div_term

        PE[:,1::2]=torch.cos(ev)
        PE[:,0::2]=torch.sin(ev)
        self.register_buffer('PE', PE.unsqueeze(0))  # 添加 batch 维度 -> (1, max_seq_len, d_model)

    def forward(self, X):

        seq_len = X.size(1)
        
        return X + self.PE[:, :seq_len, :]  # 自动广播


class ffn(nn.Module):
    def __init__(self, d_model,fc) :
        super(ffn,self).__init__()
        self.relu=nn.ReLU()
        self.ffn=nn.Linear(d_model,fc)
        self.ffn1=nn.Linear(fc,d_model)
        self.norm=nn.LayerNorm(d_model)

    def forward(self,x):
        out1=self.ffn1(self.relu(self.ffn(x)))
        return out1

class multi_att(nn.Module):
    def __init__(self,d_model,num_heads):
        super(multi_att, self).__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.d_k=d_model//num_heads

        #qkv
        self.q=nn.Linear(d_model,d_model)
        self.k=nn.Linear(d_model,d_model)
        self.v=nn.Linear(d_model,d_model)
        self.o=nn.Linear(d_model,d_model)

    def split_heads(self, x):
        B, L, D = x.shape
        x = x.view(B, L, self.num_heads, self.d_k)
        return x.transpose(1, 2)  # [B, H, L, d_k]


    def att(self,Q,K,V,mask=None):
        att=torch.matmul(Q,K.permute(0,1,3,2))/torch.sqrt(torch.tensor(self.d_k))
        if mask is not None:
            att = att.masked_fill(mask == 0, -1e9)#0的时候是-1e9
        att_score=torch.matmul(torch.softmax(att,-1),V)
       
        B, H, L, d_k = att_score.shape
        att_score = att_score.transpose(1, 2).contiguous()  # [B, L, H, d_k]
        final_com = att_score.view(B, L, -1)
        final_com=att_score.reshape(att_score.shape[0],att_score.shape[1],-1)
        return final_com


    def forward(self,qq,kk,vv,mask=None):
        Q=self.split_heads(self.q(qq))
        K=self.split_heads(self.k(kk))
        V=self.split_heads(self.v(vv))

        xxx=self.att(Q,K,V)
        return self.o(xxx)

class encoder(nn.Module):
    def __init__(self,d_model,num_heads,dropout,fc):
        super(encoder, self).__init__()
  

        self.norm1 = nn.LayerNorm(d_model)  # 层归一化
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)  # Dropout

        self.mt=multi_att(d_model,num_heads)
        self.ffn=ffn(d_model,fc) 
        #
    def forward(self,x,mask):
        att=self.mt(x,x,x,mask)
        out1=self.norm1(self.dropout(att)+x)
        fin=self.norm2(out1+self.dropout(self.ffn(out1)))
        return fin


class decoder(nn.Module):
    def __init__(self,d_model,dropout,num_heads,fc):
        super(decoder, self).__init__()
        self.norm1 = nn.LayerNorm(d_model)  # 层归一化
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)  # Dropout

        self.mtt=multi_att(d_model,num_heads)
        self.ffn=ffn(d_model,fc) 
    def forward(self,x,enc,mask_e,mask_d):
        att=self.mtt(x,x,x,mask_e)
        out1=self.norm1(self.dropout(att)+x)

        att2=self.mtt(out1,enc,enc,mask_d)
        att2=self.norm2(out1+self.dropout(att2))

        fff=self.norm3(att2+self.dropout(self.ffn(att2)))

        return  fff



class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)  # 编码器词嵌入
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)  # 解码器词嵌入
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)  # 位置编码

        # 编码器和解码器层
        self.encoder_layers = nn.ModuleList([encoder(d_model,num_heads,dropout, d_ff) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([decoder(d_model,dropout,num_heads, d_ff) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)  # 最终的全连接层
        self.dropout = nn.Dropout(dropout)  # Dropout

    def generate_mask(self, src, tgt):
        # 源掩码：屏蔽填充符（假设填充符索引为0）
        # 形状：(batch_size, 1, 1, seq_length)
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
   
        # 目标掩码：屏蔽填充符和未来信息
        # 形状：(batch_size, 1, seq_length, 1)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        # 生成上三角矩阵掩码，防止解码时看到未来信息
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask  # 合并填充掩码和未来信息掩码
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        # 生成掩码
        src_mask, tgt_mask = self.generate_mask(src, tgt)
       
        # 编码器部分
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)
       
        # 解码器部分
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))
        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)
       
        # 最终输出
        output = self.fc(dec_output)
        return output

In [63]:
# 超参数
src_vocab_size = 5000  # 源词汇表大小
tgt_vocab_size = 5000  # 目标词汇表大小
d_model = 512  # 模型维度
num_heads = 8  # 注意力头数量
num_layers = 6  # 编码器和解码器层数
d_ff = 2048  # 前馈网络内层维度
max_seq_length = 100  # 最大序列长度
dropout = 0.1  # Dropout 概率

# 初始化模型
transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# 生成随机数据
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # 源序列
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # 目标序列

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss(ignore_index=0)  # 忽略填充部分的损失
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

# 训练循环
transformer.train()
for epoch in range(100):
    optimizer.zero_grad()  # 清空梯度，防止累积
    
    # 输入目标序列时去掉最后一个词（用于预测下一个词）
    output = transformer(src_data, tgt_data[:, :-1])  
    
    # 计算损失时，目标序列从第二个词开始（即预测下一个词）
    # output形状: (batch_size, seq_length-1, tgt_vocab_size)
    # 目标形状: (batch_size, seq_length-1)
    loss = criterion(
        output.contiguous().view(-1, tgt_vocab_size), 
        tgt_data[:, 1:].contiguous().view(-1)
    )
    
    loss.backward()        # 反向传播
    optimizer.step()       # 更新参数
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.68264389038086
Epoch: 2, Loss: 8.559000015258789


KeyboardInterrupt: 

In [3]:
def generate_mask(self, src, tgt):
    # 源掩码：屏蔽填充符（假设填充符索引为0）
    # 形状：(batch_size, 1, 1, seq_length)
    src_mask = (src != 0).unsqueeze(1).unsqueeze(2)

    # 目标掩码：屏蔽填充符和未来信息
    # 形状：(batch_size, 1, seq_length, 1)
    tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
    seq_length = tgt.size(1)
    # 生成上三角矩阵掩码，防止解码时看到未来信息
    nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
    tgt_mask = tgt_mask & nopeak_mask  # 合并填充掩码和未来信息掩码
    return src_mask, tgt_mask

In [ ]:
import torch

# 设置随机种子以确保结果可复现
torch.manual_seed(42)

# 定义最大序列长度
max_seq_length = 5

# 创建一个批次大小为 2 的简单源序列和目标序列
src_data = torch.tensor([
    [2, 3, 4, 0, 0],  # 第一个样本，后两个是填充符
    [2, 3, 4, 5, 6]   # 第二个样本，没有填充符
])

tgt_data = torch.tensor([
    [1, 2, 3, 0, 0],  # 第一个样本，后两个是填充符
    [1, 2, 3, 4, 5]   # 第二个样本，没有填充符
])
def generate_mask(src, tgt):
    # 源掩码：屏蔽填充符（假设填充符索引为0）
    #在计算注意力阶段，由于attention的维度是[B, H, src_len, src_len]，因此我们也需要一个这样的mask
    #所以需要把encoder的拓展到[B, 1, 1, src_len]，这样利用广播机制，可以把剩下两个位置也加上
    src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
    
    # 目标掩码：屏蔽填充符和未来信息
    #padding 必须屏蔽“列”（key） —— 因为 padding token 不该作为 key 被其他词注意！,因此广播机制应该是对列广播
    tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)#这里之所以这样，是为了方便和后续的nopeak_mask合并，利用广播机制


    seq_length = tgt.size(1)
    nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
    tgt_mask = tgt_mask & nopeak_mask  # 合并填充掩码和未来信息掩码
    
  
    return src_mask, tgt_mask


src_mask, tgt_mask = generate_mask(src_data, tgt_data)
